In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from math import sin, cos, sqrt, atan2, radians
import xml.etree.ElementTree as ET

In [2]:
station_xml = ET.parse('bikeStations.xml')
root = station_xml.getroot()

In [3]:
id_match = {}
for station in root.findall('station'):
    terminalName = station.find('terminalName').text
    station_id = station.find('id').text
    id_match[int(station_id)] = terminalName

In [4]:
data_2015 = pd.read_csv("hubway_Trips_2015.csv", index_col=0)
data_2015.head()

/Users/shiyunqiu/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/shiyunqiu/anaconda3/envs/ipykernel_py3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,542,2015-01-01 00:21:44,2015-01-01 00:30:47,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075,277,Subscriber,1984,1
1,438,2015-01-01 00:27:03,2015-01-01 00:34:21,80,MIT Stata Center at Vassar St / Main St,42.361962,-71.092053,95,Cambridge St - at Columbia St / Webster Ave,42.372969,-71.094445,648,Subscriber,1985,1
2,254,2015-01-01 00:31:31,2015-01-01 00:35:46,91,One Kendall Square at Hampshire St / Portland St,42.366277,-71.091690,68,Central Square at Mass Ave / Essex St,42.36507,-71.1031,555,Subscriber,1974,1
3,432,2015-01-01 00:53:46,2015-01-01 01:00:58,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075,1307,Subscriber,1987,1
4,735,2015-01-01 01:07:06,2015-01-01 01:19:21,105,Lower Cambridgeport at Magazine St/Riverside Rd,42.356954,-71.113687,88,Inman Square at Vellucci Plaza / Hampshire St,42.374035,-71.101427,177,Customer,1986,2


In [5]:
data_2015['starttime_dateobj'] = np.array([datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in data_2015.starttime.values])
data_2015['stoptime_dateobj'] = np.array([datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in data_2015.stoptime.values])

In [6]:
data_2015['month'] =  np.array([i.month for i in data_2015.starttime_dateobj])
data_2015['week'] = np.array([i.week for i in data_2015.starttime_dateobj])

In [7]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [8]:
data_2015['start station latitude'] = pd.to_numeric(data_2015['start station latitude'], errors='coerce')
data_2015['start station longitude'] = pd.to_numeric(data_2015['start station longitude'], errors='coerce')
data_2015['end station latitude'] = pd.to_numeric(data_2015['end station latitude'], errors='coerce')
data_2015['end station longitude'] = pd.to_numeric(data_2015['end station longitude'], errors='coerce')

In [9]:
data_2015['distance'] = np.array([calculate_distance(data_2015.iloc[i, :]['start station latitude'],
                                                    data_2015.iloc[i, :]['start station longitude'],
                                                    data_2015.iloc[i, :]['end station latitude'],
                                                    data_2015.iloc[i, :]['end station longitude']) for i in range(data_2015.shape[0])])

In [10]:
data_2015['start_station_id'] = data_2015['start station id'].map(id_match)
data_2015['end_station_id'] = data_2015['end station id'].map(id_match)
data_2015.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,usertype,birth year,gender,starttime_dateobj,stoptime_dateobj,month,week,distance,start_station_id,end_station_id
0,542,2015-01-01 00:21:44,2015-01-01 00:30:47,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,...,Subscriber,1984,1,2015-01-01 00:21:44,2015-01-01 00:30:47,1,1,1.753865,M32029,NaN
1,438,2015-01-01 00:27:03,2015-01-01 00:34:21,80,MIT Stata Center at Vassar St / Main St,42.361962,-71.092053,95,Cambridge St - at Columbia St / Webster Ave,42.372969,...,Subscriber,1985,1,2015-01-01 00:27:03,2015-01-01 00:34:21,1,1,1.239971,M32005,NaN
2,254,2015-01-01 00:31:31,2015-01-01 00:35:46,91,One Kendall Square at Hampshire St / Portland St,42.366277,-71.091690,68,Central Square at Mass Ave / Essex St,42.365070,...,Subscriber,1974,1,2015-01-01 00:31:31,2015-01-01 00:35:46,1,1,0.947272,M32002,NaN
3,432,2015-01-01 00:53:46,2015-01-01 01:00:58,115,Porter Square Station,42.387995,-71.119084,96,Cambridge Main Library at Broadway / Trowbridg...,42.373379,...,Subscriber,1987,1,2015-01-01 00:53:46,2015-01-01 01:00:58,1,1,1.753865,M32029,NaN
4,735,2015-01-01 01:07:06,2015-01-01 01:19:21,105,Lower Cambridgeport at Magazine St/Riverside Rd,42.356954,-71.113687,88,Inman Square at Vellucci Plaza / Hampshire St,42.374035,...,Customer,1986,2,2015-01-01 01:07:06,2015-01-01 01:19:21,1,1,2.150553,M32022,NaN


In [11]:
# df = data_2015.groupby(['month','bikeid']).size().reset_index(name='counts')
# df_tmp = df.sort_values(by=['counts'])
# df_tmp

In [12]:
# df = data_2015.groupby(['week','bikeid']).size().reset_index(name='counts')
# df_tmp = df.sort_values(by=['counts'])
# df_tmp

In [22]:
station = pd.read_csv("Hubway_Stations_2011_2016.csv")
station_municipal = {}
for i in range(station.shape[0]):
    station_municipal[station.loc[i, 'Station ID']] = station.loc[i, 'Municipality']

In [23]:
data_2015['start_region'] = data_2015['start_station_id'].map(station_municipal)
data_2015['end_region'] = data_2015['end_station_id'].map(station_municipal)

In [24]:
columns_needed = ['bikeid', 'month', 'week', 'tripduration', 'distance', 'start_station_id',
                  'start station name', 'end station name', 'starttime', 'stoptime',
                  'start_region', 'end_region']

In [25]:
data_2015_cleaned = data_2015[columns_needed]
column_names = ['bikeid', 'month', 'week', 'duration', 'distance', 'start_station_id',
                'start_station', 'end_station', 'starttime', 'stoptime', 'start_region', 'end_region']
data_2015_cleaned.columns = column_names
data_2015_cleaned.head()

,bikeid,month,week,duration,distance,start_station_id,start_station,end_station,starttime,stoptime,start_region,end_region
0,277,1,1,542,1.753865,M32029,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,2015-01-01 00:21:44,2015-01-01 00:30:47,Cambridge,NaN
1,648,1,1,438,1.239971,M32005,MIT Stata Center at Vassar St / Main St,Cambridge St - at Columbia St / Webster Ave,2015-01-01 00:27:03,2015-01-01 00:34:21,Cambridge,NaN
2,555,1,1,254,0.947272,M32002,One Kendall Square at Hampshire St / Portland St,Central Square at Mass Ave / Essex St,2015-01-01 00:31:31,2015-01-01 00:35:46,Cambridge,NaN
3,1307,1,1,432,1.753865,M32029,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,2015-01-01 00:53:46,2015-01-01 01:00:58,Cambridge,NaN
4,177,1,1,735,2.150553,M32022,Lower Cambridgeport at Magazine St/Riverside Rd,Inman Square at Vellucci Plaza / Hampshire St,2015-01-01 01:07:06,2015-01-01 01:19:21,Cambridge,NaN


In [19]:
data_2015_cleaned.to_csv("trips_2015.csv", index=False)

In [ ]:
# data_2015_cleaned = pd.read_csv("trips_2015.csv")

In [12]:
# average distance by month
np.mean(data_2015_cleaned.groupby(['bikeid', 'month']).sum()['distance'].values)

167.83713358130848

In [13]:
# average distance by week
np.mean(data_2015_cleaned.groupby(['bikeid', 'week']).sum()['distance'].values)

42.610297756321785